<a href="https://colab.research.google.com/github/jcestevezc/Machine-Learning-Techniques/blob/master/Laboratorio%204/Laboratorio_IV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<img src="https://cursos.virtual.uniandes.edu.co/isis4219/wp-content/uploads/sites/162/2014/11/cropped-misisheader.png" ><br>
# Machine Learning Techniques - ISIS4219

Intersemestral 2020




## Integrantes

*   Juan José Villegas
*   Juan Sebastián Méndez



## Taller
## Segundo Problema

Ante la gran demanda de noticias que se generan hoy en día, el servicio público de radio y televisión Británica BBC desea realizar un sistema que le permita clasificar diferentes noticias para mostrarlas en su página web. Para ello, le ha brindado las noticias previamente clasificadas en cinco categorías. <br>
La idea es realizar un clasificador Näive Bayes para la realización de este problema.


# 1. Importando las librerias requeridas

In [1]:
import pandas as pd
import numpy as np
import contractions
import inflect
import re, string, unicodedata

from pandas_profiling import ProfileReport
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix, plot_precision_recall_curve
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer


In [14]:
import nltk
# Download punkt if not installed
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jjvillegas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jjvillegas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jjvillegas/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

# 2. Perfilamiento y entendimiento de los datos

## 2.1 Adquisición de datos

In [2]:
# Usamos la función load_files de sklearn para cargar todas las noticias en sus respectivas categorías.
news_bunch = load_files("./Punto 2/Noticias BBC", load_content=True, encoding='utf-8', decode_error='ignore')
categories = news_bunch.target_names
categories

['business', 'entertainment', 'politics', 'sport', 'tech']

In [3]:
# Transformamos los datos obtenidos por load_files en un dataframe de pandas.
news = pd.DataFrame(news_bunch.data, news_bunch.target, columns=["text"])
news.reset_index(level=0, inplace=True)
news['class'] = news['index'].apply(lambda x: categories[x])
del news['index']

news['word_count'] = [len(x) for x in news['text']]
news['mode'] = [max(set(x.split(' ')), key = x.split(' ').count) for x in news['text']]
news['max'] = [[max([len(x) for x in i.split(' ')])][0] for i in news['text']]
news['min'] = [[min([len(x) for x in i.split(' ')])][0] for i in news['text']]

news.head()

,text,class,word_count,mode,max,min
0,Tate & Lyle boss bags top award\n\nTate & Lyle...,business,1118,the,19,1
1,Halo 2 sells five million copies\n\nMicrosoft ...,tech,1846,the,21,1
2,MSPs hear renewed climate warning\n\nClimate c...,politics,1731,the,22,1
3,Pavey focuses on indoor success\n\nJo Pavey wi...,sport,775,the,13,1
4,Tories reject rethink on axed MP\n\nSacked MP ...,politics,3298,the,18,1


In [4]:
ProfileReport(news)

Render HTML: 100%|██████████| 1/1 [00:00<00:00,  2.93it/s]


# 3. Preparación de datos

Al igual que en el laboratorio, vamos a hacer los pasos estándar para limpiar los datos y dejarlos listos para nuestro algoritmo:

 1. Eliminamos contracciones usando "Contractions" (Por ejemplo, i'm -> I am). Un programa en python que lo hace por nosotros.
 1. Luego hacemos una tokenización de los textos. Es decir, hacemos una lista con todas las palabras encontradas en cada texto.
 1. Luego hacemos un preprocesamiento de los datos. Esto incluye:
    1. Remover caractéres que no son ASCII.
    1. Pasar todas las palabras a minúsculas.
    1. Remover puntuación.
    1. Reemplazar números por su representación en palabras.
    1. Quitar las stopwords en inglés.
 

In [4]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def preproccesing(words):
    words = to_lowercase(words)
    words = replace_numbers(words)
    words = remove_punctuation(words)
    words = remove_non_ascii(words)
    words = remove_stopwords(words)
    return words

In [5]:
news['text'] = news['text'].apply(contractions.fix) #Aplica la corrección de las contracciones
news['words'] = news['text'].apply(word_tokenize)
news['words'] = news['words'].apply(preproccesing) #Aplica la eliminación del ruido
news.head()

,text,class,word_count,mode,max,min,words
0,Tate & Lyle boss bags top award\n\nTate & Lyle...,business,1118,the,19,1,"[tate, lyle, boss, bags, top, award, tate, lyl..."
1,Halo 2 sells five million copies\n\nMicrosoft ...,tech,1846,the,21,1,"[halo, two, sells, five, million, copies, micr..."
2,MSPs hear renewed climate warning\n\nClimate c...,politics,1731,the,22,1,"[msps, hear, renewed, climate, warning, climat..."
3,Pavey focuses on indoor success\n\nJo Pavey wi...,sport,775,the,13,1,"[pavey, focuses, indoor, success, jo, pavey, m..."
4,Tories reject rethink on axed MP\n\nSacked MP ...,politics,3298,the,18,1,"[tories, reject, rethink, axed, mp, sacked, mp..."


Ahora que los datos están un poco más limpios, nos preparamos para hacer Stemming y Lematización. Para esto utilizamos soluciones estándar como el LancasterStemmer, y el WordNetLemmatizer.

In [6]:
def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def stem_and_lemmatize(words):
    stems = stem_words(words)
    lemmas = lemmatize_verbs(words)
    return stems + lemmas

news['words'] = news['words'].apply(stem_and_lemmatize) #Aplica lematización y Eliminación de Prefijos y Sufijos.
news.head()

,text,class,word_count,mode,max,min,words
0,Tate & Lyle boss bags top award\n\nTate & Lyle...,business,1118,the,19,1,"[tat, lyl, boss, bag, top, award, tat, lyl, ch..."
1,Halo 2 sells five million copies\n\nMicrosoft ...,tech,1846,the,21,1,"[halo, two, sel, fiv, mil, cop, microsoft, cel..."
2,MSPs hear renewed climate warning\n\nClimate c...,politics,1731,the,22,1,"[msps, hear, renew, clim, warn, clim, chang, c..."
3,Pavey focuses on indoor success\n\nJo Pavey wi...,sport,775,the,13,1,"[pavey, focus, indo, success, jo, pavey, miss,..."
4,Tories reject rethink on axed MP\n\nSacked MP ...,politics,3298,the,18,1,"[tory, reject, rethink, ax, mp, sack, mp, howa..."


In [8]:
X, y = news['words'],news['class']

In [9]:
news['words'] = news['words'].apply(lambda x: ' '.join(map(str, x)))
news

,text,class,word_count,mode,max,min,words
0,Tate & Lyle boss bags top award\n\nTate & Lyle...,business,1118,the,19,1,tat lyl boss bag top award tat lyl chief execu...
1,Halo 2 sells five million copies\n\nMicrosoft ...,tech,1846,the,21,1,halo two sel fiv mil cop microsoft celebr bump...
2,MSPs hear renewed climate warning\n\nClimate c...,politics,1731,the,22,1,msps hear renew clim warn clim chang could com...
3,Pavey focuses on indoor success\n\nJo Pavey wi...,sport,775,the,13,1,pavey focus indo success jo pavey miss janu vi...
4,Tories reject rethink on axed MP\n\nSacked MP ...,politics,3298,the,18,1,tory reject rethink ax mp sack mp howard fligh...
...,...,...,...,...,...,...,...
2220,EU fraud clampdown urged\n\nEU member states a...,politics,2470,the,17,1,eu fraud clampdown urg eu memb stat fail repor...
2221,Tindall aiming to earn Lions spot\n\nBath and ...,sport,1232,to,13,1,tindal aim earn lion spot bath england cent mi...
2222,Jamelia's return to the top\n\nR&B star Jameli...,entertainment,2196,the,19,1,jamel return top r b star jamel three brit nom...
2223,Oscars steer clear of controversy\n\nThe Oscar...,entertainment,3801,the,21,1,osc ste clear controversy osc nomin list left ...


Se optienen los dummmies de las clases

In [11]:
y = pd.get_dummies(y)
y

,business,entertainment,politics,sport,tech
0,1,0,0,0,0
1,0,0,0,0,1
2,0,0,1,0,0
3,0,0,0,1,0
4,0,0,1,0,0
...,...,...,...,...,...
2220,0,0,1,0,0
2221,0,0,0,1,0
2222,0,1,0,0,0
2223,0,1,0,0,0


Ahora se aplica la transformación. Escogimos, CountVectorized ya que rindió mejor con Naive Bayes 

In [13]:
X_vectorized = CountVectorizer(stop_words='english').fit_transform(X)
print(X_vectorized.shape)
X_vectorized.toarray()[0]

(2225, 36512)


array([0, 0, 0, ..., 0, 0, 0])

# 4. Modelamiento

Para encontrar el mejor modelo, vamos a hacer un gridSearch haciendo uso de tres formas de hacer la matriz de palabras: tfidf, countVectorizer, y HashingVectorizer. También vamos a probar con diferentes variaciones de Naive Bayes con el fin de encontrar combinación que rinda mejor.

In [21]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, BernoulliNB

estimators = [
    ('transform', CountVectorizer(stop_words='english')), 
    ('clf', BernoulliNB())
]
pipe = Pipeline(estimators)

param_grid = {
    'transform': [TfidfVectorizer(stop_words='english'), CountVectorizer(stop_words='english'), HashingVectorizer(stop_words='english', n_features=2**13)],
    'clf': [BernoulliNB(), GaussianNB(), MultinomialNB(), ComplementNB()]
}

grid = GridSearchCV(pipe, param_grid=param_grid, verbose = 20, n_jobs=-1, error_score=0.0)
grid.fit(X_train, y_train)
best_estimator = grid.best_estimator_
best_estimator

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   

Pipeline(memory=None,
         steps=[('transform',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words='english', strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [28]:
grid.best_score_

0.9735955056179776

Como se puede ver, el mejor pipeline para este problema de clasificación parece ser transformando la data con un CountVector junto a un clasificador Multinomial Naive Bayes (MultinomialNB) dando un best score de 0.9735 en training. 

# 5. Predicción

A continuación, se hace la predicción para poder después poder validar

In [15]:
y.shape[1]

5

In [23]:
y_predic_best = best_estimator.predict(X_test)

In [24]:
y_predic_best

array(['sport', 'sport', 'entertainment', 'business', 'business', 'tech',
       'business', 'sport', 'tech', 'politics', 'politics', 'politics',
       'sport', 'politics', 'business', 'entertainment', 'tech',
       'politics', 'politics', 'politics', 'entertainment', 'business',
       'politics', 'politics', 'business', 'tech', 'politics',
       'entertainment', 'business', 'tech', 'politics', 'sport', 'tech',
       'tech', 'sport', 'politics', 'sport', 'tech', 'sport', 'sport',
       'tech', 'entertainment', 'tech', 'politics', 'business',
       'politics', 'business', 'sport', 'sport', 'politics', 'politics',
       'business', 'tech', 'business', 'business', 'politics', 'sport',
       'entertainment', 'politics', 'entertainment', 'sport', 'business',
       'entertainment', 'business', 'entertainment', 'entertainment',
       'business', 'entertainment', 'entertainment', 'sport', 'sport',
       'sport', 'sport', 'sport', 'entertainment', 'sport', 'tech',
       'politics',

# 6. Validación

Se saca un reporte de clasificación para ver la matriz de confusión de las diferentes clases ya contra el conjunto de test

In [27]:
report_best_estimator = classification_report(y_test, y_predic_best, target_names=categories)
print("Test Report for Näive Bayes\n", report_best_estimator)

Test Report for Näive Bayes
                precision    recall  f1-score   support

     business       0.99      0.98      0.98       100
entertainment       0.97      0.94      0.95        77
     politics       0.95      0.99      0.97        85
        sport       1.00      1.00      1.00       107
         tech       0.96      0.97      0.97        76

     accuracy                           0.98       445
    macro avg       0.98      0.98      0.98       445
 weighted avg       0.98      0.98      0.98       445



# Conclusiones

* GridSearch es una herramienta muy poderosa. Deja también probar diferentes combinaciones de pipelines para encontrar el mejor match de transformación vs clasificación.
* Creíamos que TF-IDF iba a tener mejor rendimiento pero al perecer para este problema en especial, el CountVector fue suficiente. Seguro no es necesario el contexto del resto de los artículos para clasificar uno de ellos
* Clasificar noticias de deportes es extrañamente preciso. Seguro con una muestra más grande y diversa de test, va a ser posible ver en realidad su rendimiento
* Es muy rápido correr Naive Bayes y por lo tanto, se puede iterar mucho más rápido sobre el modelo
* Todo el proceso de preparación de los datos, tiende a ser muy estándar. Igual hay que tener cuidado donde el contexto semántico sea muy importante, de no estar quitando palabras que nos estén dando información relevante para el problema.

# Ahora: Utilizando deep learning

In [55]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.base import BaseEstimator, TransformerMixin

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import activations
from tensorflow.keras import metrics

In [27]:
y.shape

(2225, 5)

In [54]:
X.shape

(2225,)

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.2, random_state=33)

In [84]:
X_vectorized.shape[1]

36512

In [89]:
def build_model(output=5, n_hidden=1, n_neurons_input=512, n_neurons_hidden=30, dropout=0.1, activation='relu'):
    model = models.Sequential()
    
    model.add(layers.Dense(n_neurons_input, activation=activation, name='Input_layer'))
    #model.add(layers.Input(shape=(1, n_neurons_input), name='Input_layer'))
    for layer in range(n_hidden):
        name = 'Hidden_layer_%d' % layer
        model.add(layers.Dense(n_neurons_hidden, activation=activation, name=name))
    
    model.add(layers.Dropout(dropout, name='Dropout_layer'))
    model.add(layers.Dense(output, activation=activations.softmax, name='Output_layer'))
    model.compile(
        loss=losses.categorical_crossentropy, 
        optimizer=optimizers.SGD(), 
        metrics=['accuracy'])
    
    return model

modelCV = KerasClassifier(build_fn=build_model, epochs=20, batch_size=20, verbose=1, shuffle=True)

In [94]:
early_stopping_callback = callbacks.EarlyStopping(monitor='loss', patience=3)

In [95]:
# Matrix to array tranformer

class MatrixToArrayTransformer(BaseEstimator):

    def __init__(self):
        return
    
    def fit(self, X, y=None, **kwargs):
        return self
    
    def transform(self, X, **kwargs):
        return X.toarray()

In [109]:
pipeline = Pipeline([
    ('transform', CountVectorizer(stop_words='english')),
    ('matrix', MatrixToArrayTransformer()),
    ('model', modelCV)
])

param_grid = {
    'transform': [TfidfVectorizer(stop_words='english'), CountVectorizer(stop_words='english')],
    'model__n_hidden': [1, 2],
    'model__n_neurons_input': [512, 10000, 100],
    'model__n_neurons_hidden': [512, 10000, 100],
    'model__dropout': [0.1, 0.5, 0.3]
}

grid = RandomizedSearchCV(pipeline, param_grid, n_iter=10, cv=3, random_state=20, verbose=1)

In [110]:
grid.fit(X_train, y_train, model__callbacks=[early_stopping_callback])

===========] - 23s 375ms/step - loss: 1.5422 - accuracy: 0.4545
Epoch 12/20
60/60 [==============================] - 23s 376ms/step - loss: 1.5339 - accuracy: 0.4621
Epoch 13/20
60/60 [==============================] - 22s 375ms/step - loss: 1.5247 - accuracy: 0.4815
Epoch 14/20
60/60 [==============================] - 23s 376ms/step - loss: 1.5149 - accuracy: 0.4933
Epoch 15/20
60/60 [==============================] - 23s 375ms/step - loss: 1.5035 - accuracy: 0.4907
Epoch 16/20
60/60 [==============================] - 23s 376ms/step - loss: 1.4924 - accuracy: 0.5270
Epoch 17/20
60/60 [==============================] - 23s 377ms/step - loss: 1.4793 - accuracy: 0.5632
Epoch 18/20
60/60 [==============================] - 23s 377ms/step - loss: 1.4660 - accuracy: 0.5556
Epoch 19/20
60/60 [==============================] - 23s 376ms/step - loss: 1.4510 - accuracy: 0.6425
Epoch 20/20
30/30 [==============================] - 1s 44ms/step - loss: 1.4349 - accuracy: 0.6684
Epoch 1/20
60/60 [==

RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('transform',
                                              CountVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.int64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=1.0,
                                                              max_features=None,
                                                              min_df=1,
                                                     

In [111]:
best_model = grid.best_estimator_

In [115]:
grid.best_params_

{'transform': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=1.0, max_features=None, min_df=1,
                 ngram_range=(1, 1), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None),
 'model__n_neurons_input': 10000,
 'model__n_neurons_hidden': 100,
 'model__n_hidden': 2,
 'model__dropout': 0.5}

In [112]:
best_model['model'].model.summary()

Model: "sequential_38"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_layer (Dense)          multiple                  332060000 
_________________________________________________________________
Hidden_layer_0 (Dense)       multiple                  1000100   
_________________________________________________________________
Hidden_layer_1 (Dense)       multiple                  10100     
_________________________________________________________________
Dropout_layer (Dropout)      multiple                  0         
_________________________________________________________________
Output_layer (Dense)         multiple                  505       
Total params: 333,070,705
Trainable params: 333,070,705
Non-trainable params: 0
_________________________________________________________________


In [106]:
best_model['model'].model

## Predicción

In [113]:
y_pred = best_model.predict(X_test)
y_pred

23/23 [==============================] - 1s 60ms/step


array([3, 3, 1, 0, 0, 4, 0, 3, 4, 1, 2, 2, 3, 2, 0, 1, 4, 2, 2, 2, 1, 0,
       1, 2, 0, 4, 2, 1, 0, 4, 2, 3, 4, 4, 3, 2, 3, 4, 3, 3, 4, 1, 4, 2,
       0, 2, 0, 3, 3, 2, 2, 0, 4, 0, 0, 2, 3, 1, 2, 1, 3, 0, 1, 0, 1, 1,
       0, 1, 1, 3, 3, 3, 3, 3, 1, 3, 4, 2, 3, 2, 0, 0, 3, 3, 1, 3, 2, 0,
       2, 4, 3, 0, 0, 3, 1, 2, 0, 3, 4, 2, 4, 0, 0, 3, 1, 2, 0, 2, 3, 1,
       3, 2, 4, 3, 4, 2, 2, 0, 0, 3, 1, 0, 4, 2, 0, 2, 0, 3, 0, 2, 0, 3,
       0, 2, 0, 0, 0, 0, 4, 1, 4, 4, 1, 1, 0, 4, 1, 2, 2, 1, 1, 3, 3, 0,
       4, 0, 0, 0, 1, 1, 0, 1, 1, 3, 3, 2, 4, 0, 3, 3, 1, 1, 2, 2, 0, 4,
       3, 3, 1, 0, 4, 1, 0, 2, 1, 1, 2, 4, 4, 4, 3, 0, 1, 0, 0, 1, 1, 4,
       0, 3, 2, 4, 2, 1, 3, 0, 3, 1, 3, 1, 1, 1, 4, 3, 0, 4, 4, 3, 3, 4,
       3, 4, 0, 0, 3, 0, 0, 2, 4, 2, 0, 4, 1, 4, 4, 1, 3, 2, 0, 3, 3, 0,
       4, 3, 3, 4, 3, 1, 3, 4, 3, 0, 3, 0, 3, 2, 3, 3, 0, 2, 4, 4, 1, 2,
       4, 0, 4, 4, 1, 4, 0, 1, 2, 3, 0, 4, 0, 2, 2, 2, 3, 3, 0, 4, 2, 2,
       0, 2, 1, 3, 3, 3, 1, 0, 1, 3, 1, 3, 3, 1, 0,

## Validación

In [114]:
Y_test = y_test.argmax(axis=-1)
print('Reporte para el modelo entonado')
print(classification_report(Y_test, y_pred, target_names=y.columns))

Reporte para el modelo entonado
               precision    recall  f1-score   support

     business       0.96      0.96      0.96       100
entertainment       0.99      0.97      0.98        77
     politics       0.96      0.93      0.95        85
        sport       0.99      1.00      1.00       107
         tech       0.96      1.00      0.98        76

     accuracy                           0.97       445
    macro avg       0.97      0.97      0.97       445
 weighted avg       0.97      0.97      0.97       445

